In [494]:
#CRAWLING DATA FROM YAHOO MOVIES
import requests
import re
from bs4 import BeautifulSoup
import json

Y_MOVIE_URL = "https://movies.yahoo.com.tw/movieinfo_main/"

class MovieCrawler(object):

    def __init__(self):
        self.session = requests.Session()
        
    def get_movies(self, page_url):
        movies = []
          # with open(page_url,'r') as f:
          #   contents = f.read()
        for i in range(2,3):
            response = self.session.get(page_url+str(i))#+"?page="+str(i))
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')
            movie_list = soup.find_all('div', id='content_l')
            # print(movie_list)
            for movie in movie_list:
                label_class = []
                # print(movie)
                try:
                    ch_name = movie.find('div', class_='movie_intro_info_r').h1#.text.strip()
                    try:
                        ch_name.span.decompose()
                    except:
                        pass
                    ch_name = ch_name.text.strip()
                    en_name = movie.find('div', class_='movie_intro_info_r').h3.text.strip()
                    release_date = movie.find('div', class_='movie_intro_info_r').find('span',class_=None).text[5:].strip()#.span.text.split('上映日期：')[-1].strip()
                    labels = movie.find_all('div',class_ = 'level_name')
                    for label in labels:
                        try:
                            lb = label.find('a',class_='gabtn').text.strip()
                        except:
                            break
                        label_class.append(lb)
                    intro = movie.find('div', class_='gray_infobox_inner').span.text.strip().replace('\n','')
                    doc_id = i
                    movie_info = {
                                'doc_id' : doc_id,
                                'ch_name': ch_name,
                                'en_name': en_name,
                                'label[class]': label_class,
                                'release_date': release_date,
                                'intro': intro}
                    movies.append(movie_info)
                except:
                    print(i)
                    continue
        return movies
    
    def savejson(self,data):
        with open("./test.json", "w",encoding = 'utf-8') as final:
            json.dump(data, final,ensure_ascii=False)
        final.close()

    def addjson(self,data):
        with open("./test.json", "r",encoding='utf-8') as fp:
            listObj = json.load(fp)
            for m in data:
                listObj.append(m)
        print(len(listObj))
        fp.close()
        with open("./test.json", "w",encoding = 'utf-8') as final:
            json.dump(listObj, final,ensure_ascii=False)
        final.close()
        
    def readjson(self,data):
        with open(f"./{data}.json", "r",encoding = 'utf-8') as final:
            listObj = json.load(final)
        final.close()
        return listObj
            

In [495]:
crawler = MovieCrawler()
movie = crawler.get_movies(Y_MOVIE_URL)
# crawler.savejson(movie)
listObj = crawler.readjson("hw2")
# THE RESULTS : AS THE FOLLOWING SECTION
# {'ch_name', 'en_name', 'movie_url', 'release_date', 'intro'}
# print(len(movie))
# print(*movie, sep="\n")

In [531]:

class search_engine():
        
    def query(text):
        count_all = 0
        count = 0
        movie_prl = []
        movie_list = []
        #find all movie related to text
        regex = re.compile(rf'(.*){text}')
        for movie,data in movie_data.items():
            if regex.match(dict(data)['ch_name']) or regex.match(dict(data)['en_name']) or regex.match(dict(data)['intro']):
                count_all += 1
        for movie,index in inverted_index.items():
            if regex.match(str(movie)):
                for i in index:
                    movie_list.append(i[0])
        #print all movies related with text
        movies_list=list(dict.fromkeys(movie_list))
        data = []
        for ID in movies_list:
            if regex.match(movie_data[ID]['ch_name']) or regex.match(movie_data[ID]['en_name']) or regex.match(movie_data[ID]['intro']):
                count+=1
            data.append([movie_data[ID]['doc_id'],movie_data[ID]['pagerank'],movie_data[ID]['ch_name'], movie_data[ID]['en_name'],movie_data[ID]['intro']])
        data.sort(key = lambda x:x[1],reverse = True)
        for movie in data:
            print("DOC_ID:",movie[0])
            print("CHINESE TITLE:",movie[2])
            print("ENGLISH TITLE:",movie[3])
            print("PAGERANK:",movie[1])
            print("INTRO:",movie[4])
            print("\n")
        print("Precision = "+ str(count/len(movie_list)*100)+"%")
        print("Recall = "+str(count/count_all*100)+"%")
        print(count)
        print(count_all)
        print(len(movie_list))
        

# INVERTED INDEX

In [ ]:
from collections import Counter
import networkx as nx
import jieba
import re
from zhon.hanzi import punctuation
stopwords = [line.strip().replace('\n','') for line in open('../hw1/stopwords.txt','r').readlines()]
punctuations = [line.strip().replace('\n','') for line in open('../hw1/punctuation.txt','r').readlines()]
jieba.load_userdict('../hw1/userDict.txt')
punctuation = list(punctuation)

intro_all = []
for content in listObj:
    intro = content['intro'].replace('\n','').replace('\t','').replace('\u3000','')
    intro = re.sub('[^\u4e00-\u9fa5]','',intro)
    intro = jieba.lcut(intro)
    Intro = []
    for word in intro:
        if word not in stopwords:
            Intro.append(word)
    Intro.insert(0,content['ch_name'])
    Intro.insert(0,content['doc_id'])
    intro_all.append(Intro)

words = {}
for intro in intro_all:
    for word in intro:
        if word in words:
            words[word].append(intro[0])
        else:
            words[word] = [intro[0]]
            
from collections import Counter
inverted_index = {}
for i,j in words.items():
#     print(i)
    wordcount = (Counter(j))
    list(wordcount.items()).sort(key = lambda x : x[1],reverse = True)
    wordcounts = list(wordcount.items())
    inverted_index[i] = wordcounts

# PAGERANK

In [498]:
import networkx as nx
G = nx.DiGraph()
Edges = []
for word,doc_id in inverted_index.items():
    edges = []
    for ID in doc_id[2:]:
        if ID[0] != doc_id[0][0]:
            edges.append((doc_id[0][0],ID[0]))
        Edges.append(edges)

for edges in Edges:
    for edge in edges:
        G.add_edge(edge[0],edge[1])

pagerank_list = nx.pagerank(G,alpha = 1)
# print(pagerank_list)

PageRankList = list(pagerank_list.items())
PageRankList.sort(key = lambda x:x[0])

# CREATING DOC LINKS

In [499]:
DOC_LINK = {}
doc_link = []
i = 0
for data in inverted_index.values():
    id_num = intro_all[i][0] 
    doc_id = data[0][0]
    for ID in data:
        if ID[0] not in doc_link and ID[0]!= doc_id:
            doc_link.append(ID[0])
    if doc_id != id_num:
        i+=1
        doc_link.sort()
        DOC_LINK[id_num] = doc_link
        doc_link = []
    

# SAVING DATA into json file hw2.json

In [503]:
DATA = []
for listobj in listObj:
    try:
        d_link = DOC_LINK[dict(listobj)['doc_id']]
    except:
        d_link = "None"
    try:
        prl = dict(PageRankList)[dict(listobj)['doc_id']]
    except:
        prl = 0
    info = {
        'doc_id' : dict(listobj)['doc_id'],
        'ch_name' : dict(listobj)['ch_name'],
        'en_name' : dict(listobj)['en_name'],
        'pagerank' : prl,
        'label[class]' : dict(listobj)['label[class]'],
        'intro' : dict(listobj)['intro'],
        'release_date' : dict(listobj)['release_data'],
        'links(doc_id)' : d_link,
    }
#     print(dict(listobj)['doc_id'])
    DATA.append(info)
with open("./test.json", "w",encoding = 'utf-8') as final:
    json.dump(DATA, final,ensure_ascii=False)
final.close()

# Turn data into dictionary with id as key

In [504]:
movie_data = {}
for movie in DATA:
    movie_data[dict(movie)['doc_id']] = movie

# QUERY

In [532]:
search_engine.query("玩命")

DOC_ID: 11560
CHINESE TITLE: 天崩颶裂
ENGLISH TITLE: 13 Minutes
PAGERANK: 0.0003905089254814578
INTRO: ★《天劫倒數》＋《玩命颶風》，今年唯一天災巨片！★《聖母峰》、《怒火地平線》特效團隊打造震撼災難鉅作！★ 全球知名「風暴追逐者」崔維斯法科姆參與製作與故事發想！★ 毀滅龍捲風即將來襲，四個家庭將展現永不磨滅的人性光輝！面對史上最大龍捲風來襲，13分鐘生死倒數美國史上最強大的龍捲風，即將撲向明尼內瓦這個平凡小鎮。四個家庭成員，在龍捲風警報發佈後，只有十三分鐘的時間到達避難所。他們該如何克服分歧，重新從彼此身上找到生存的意義與力量呢？關於電影：《天劫倒數》＋《玩命颶風》，今年唯一天災巨片《天崩颶裂》是由《聖母峰》（Everest）、《怒火地平線》（Deepwater Horizon）特效團隊打造的災難巨片。全片透過龍捲風來襲時，通常只有十三分鐘時間可以避難的主題，讓一個平凡小鎮上的四個家庭，在這場災難中產生交集。原本尋常的一天，卻因為災難性的驟變，讓小鎮於災後必須重建；人生在轉瞬之間，也發生翻天覆地的變化。故事發想暨製片人崔維斯法科姆（Travis Farncombe）表示：「電影不僅關注人們如何在天災的影響下倖存，更將表現人們如何在日常生活中求得生存。」這也讓電影除了震撼人心的災難場面之外，同時幅射出種族、墮胎、身障、同志與排外主義等全球議題，讓故事充滿可看性。崔維斯繼續表示：「我們賦予每位角色屬於自己的任務及挑戰，我們討論著他們想要什麼、需要什麼、目標是什麼，為什麼他們會聽從或不聽從、聽到或沒聽到暴風警告，以及他們的行為舉止是否恰當。」導演暨編劇、同時也是本片監製的琳賽葛斯林（Lindsay Gossling）補充：「我想將風暴微妙的構造與人際關係的複雜性相提並論，彰顯大自然如何不分青紅皂白、全面性的反撲。我想創造一個代表全球問題縮影的世界，並讓大自然摧毀那個世界，讓觀眾去思考重建是否將會更好。不同於風暴的平靜，片中人物也將在生活高潮的轉折點被彰顯出來。」值得一提的是，崔維斯法科姆本身就是全球知名的「風暴追逐者」。將風暴視為一生熱情的他表示：「我在年輕時，和家人在農舍小屋中經歷一場龍捲風，湖對面的幾棟小屋和拖車停車場，都因此被摧毀。那次事件，激起我對惡劣氣候的好奇心，進而